In [409]:
# import code block
from selenium import webdriver
from bs4 import BeautifulSoup
import zipfile
import pandas as pd
import requests
import csv
import time
import random
import matplotlib.pyplot as plt
from urllib.request import urlopen
import re
from pandas import ExcelWriter

In [410]:
html = urlopen('https://masslandlords.net/policy/eviction-data/?ct=t%28Event+Webinar+2021-01-22+T-21+v1%29')
bs = BeautifulSoup(html, 'html.parser')
# collect all of the dates urls into one list
urls = bs.find_all('a',{'href':re.compile('https://masslandlords.net/policy/eviction-data/filings-week-ending-*')})


In [411]:
# clean urls of unnecessary attribute data - only need href
urls = [url['href'] for url in urls]

dates=urls
dates=[e[55:77] for e in dates]

pd.DataFrame(dates, columns=['Dates'])

,Dates
0,week-ending-2020-10-24
1,week-ending-2020-10-31
2,week-ending-2020-11-07
3,week-ending-2020-11-14
4,week-ending-2020-11-21
5,week-ending-2020-11-28
6,week-ending-2020-12-05
7,week-ending-2020-12-12
8,week-ending-2020-12-19
9,week-ending-2020-12-26


# Change URL[X] to the Date of your choosing based on dataframe above

In [412]:
date=urls[0]
print(date)
html = urlopen(date)
bs = BeautifulSoup(html, 'html.parser')
text = bs.find('section',id='main-content').find('p', class_='monospace').get_text()

https://masslandlords.net/policy/eviction-data/filings-week-ending-2020-10-24/


In [413]:
a,b,c,d,e,f =text.split('--')

In [414]:
# remove beginning and ending newlines
reg = re.compile('^\r\n')
c = reg.sub('',c,count=1)
reg2 = re.compile('\r\n\r\n\r\n$')
c=reg2.sub('',c,count=1)
c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11 = c.split("\r\n\r\n\r\n")

In [415]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c1 = reg.sub('',c1,count=1)

# split the table into separate columns
c1 = c1.split('\r\n')
for index in range(1,len(c1)):
    c1[index] = re.split('  +',c1[index])

# load the data into a dataframe
courts = pd.DataFrame(c1[2:],columns=c1[1])
courts

,(n),Count,Percent
0,northeast,16,32.7%
1,southeast,8,16.3%
2,eastern,6,12.2%
3,central,6,12.2%
4,metro_south,6,12.2%
5,western,5,10.2%
6,somerville district,1,2.0%
7,taunton district,1,2.0%


In [416]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c2 = reg.sub('',c2,count=1)

# split the table into separate columns
c2 = c2.split('\r\n')
for index in range(1,len(c2)):
    c2[index] = re.split('  +',c2[index])

# load the data into a dataframe
party_type = pd.DataFrame(c2[2:],columns=c2[1])
party_type

,(n),Plaintiffs,Defendants
0,Corporate Entity,17,0
1,Natural Person,32,49
2,Total,49,49


In [417]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c3 = reg.sub('',c3,count=1)

# split the table into separate columns
c3 = c3.split('\r\n')
for index in range(0,len(c3)):
    c3[index] = re.split('  +',c3[index])

# load the data into a dataframe
party_type_pct = pd.DataFrame(c3[1:],columns=c3[0])
party_type_pct

,(%),Plaintiffs,Defendants
0,Corporate Entity,34.7%,0.0%
1,Natural Person,65.3%,100.0%
2,Total,100.0%,100.0%


In [418]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c4 = reg.sub('',c4,count=1)

# split the table into separate columns
c4 = c4.split('\r\n')
for index in range(0,len(c4)):
    c4[index] = re.split('  +',c4[index])

# load the data into a dataframe
plaintiff_repr = pd.DataFrame(c4[2:],columns=c4[1])
plaintiff_repr

,(n),Has Attorney,Pro Se,Total
0,Required,16,1,17
1,Optional,12,20,32
2,Total,28,21,49


In [419]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c5 = reg.sub('',c5,count=1)

# split the table into separate columns
c5 = c5.split('\r\n')
name = c5[0]
for index in range(0,len(c5)):
    c5[index] = re.split('  +',c5[index])

# load the data into a dataframe
plaintiff_repr_pct = pd.DataFrame(c5[1:],columns=c5[0])
plaintiff_repr_pct

,(%),Has Attorney,Pro Se,Total
0,Required,32.7%,2.0%,34.7%
1,Optional,24.5%,40.8%,65.3%
2,Total,57.1%,42.9%,100.0%


In [420]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c6 = reg.sub('',c6,count=1)

# split the table into separate columns
c6 = c6.split('\r\n')
for index in range(0,len(c6)):
    c6[index] = re.split('  +',c6[index])

# load the data into a dataframe, cutting off note: "Defendants will not have an attorney known to the plaintiff at time of filing unless this is an ongoing matter. 100% pro se is to be expected in all filing reports except to the extent time passes between filing and staff review. See outcomes report for final assessment."
defendant_repr = pd.DataFrame(c6[4:],columns=c6[3])
defendant_repr

,Required,0,0,0
0,Optional,5,44,49
1,Total,5,44,49


In [421]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c7 = reg.sub('',c7,count=1)

# split the table into separate columns
c7 = c7.split('\r\n')
for index in range(0,len(c7)):
    c7[index] = re.split('  +',c7[index])

# load the data into a dataframe
defendant_repr_pct = pd.DataFrame(c7[1:],columns=c7[0])
defendant_repr_pct

,(%),Has Attorney,Pro Se,Total
0,Required,0.0%,0.0%,0.0%
1,Optional,10.2%,89.8%,100.0%
2,Total,10.2%,89.8%,100.0%


In [422]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c8 = reg.sub('',c8,count=1)

# split the table into separate columns
c8 = c8.split('\r\n')
for index in range(len(c8)):
    c8[index] = re.split('  +',c8[index])

# load the data into a dataframe
num_adults = pd.DataFrame(c8[2:-1],columns=c8[0])
num_adults

,Number of Adults in Households,Count,Percent
0,2,10,20.4%
1,3,3,6.1%


In [423]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c9 = reg.sub('',c9,count=1)

# split the table into separate columns
c9 = c9.split('\r\n')
for index in range(len(c9)):
    c9[index] = re.split('  +',c9[index])

# load the data into a dataframe
init_action = pd.DataFrame(c9[1:],columns=c9[0])
init_action

,Initiating Action,Count,Percent
0,Cause,21,42.9%
1,Non-payment,18,36.7%
2,No Fault,5,10.2%
3,Foreclosure,3,6.1%
4,Other,2,4.1%


In [424]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c10 = reg.sub('',c10,count=1)

# split the table into separate columns
c10 = c10.split('\r\n')
for index in range(len(c10)):
    c10[index] = re.split('  +',c10[index])

# load the data into a dataframe
rate_per_100k = pd.DataFrame(c10[2:-1],columns=c10[1])
rate_per_100k

,Municipality,Residents,Count
0,Vineyard Haven,52,1
1,Osterville,28,1
2,East Falmouth,17,1
3,Topsfield,16,1
4,Shirley,13,1
5,Southbridge,11,2
6,Amesbury,6,1
7,Brockton,5,5
8,Newburyport,5,1
9,Pembroke,5,1


In [425]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c11 = reg.sub('',c11,count=1)

# split the table into separate columns
c11 = c11.split('\r\n')
for index in range(len(c11)):
    c11[index] = re.split('  +',c11[index])

# load the data into a dataframe
rate_per_100k_renters = pd.DataFrame(c11[2:-2],columns=c11[1])
rate_per_100k_renters

,County,Renter Households,Count
0,Dukes,24,1
1,Plymouth,20,9
2,Berkshire,12,2
3,Essex,11,13
4,Barnstable,10,2
5,Hampden,4,3
6,Middlesex,3,7
7,Worcester,3,4
8,Bristol,2,2
9,Suffolk,2,4


In [426]:
# remove beginning and ending newlines
reg2 = re.compile('\r\n\r\n\r\n$')
reg3=re.compile('\r\n\r\n$')

#first two dates only have two spaces while after that, they have three spaces
if date==urls[0] or date==urls[1]:
    d=reg3.sub('',d,count=1)
    d0,d1,d2,d3,d4,d5,d6=d.split("\r\n\r\n")
    
else:
    d=reg2.sub('',d,count=1)
    d1,d2= d.split("\r\n\r\n\r\n")
    d1,d2,d3,d4,d5,d6,d7=d2.split("\r\n\r\n")

In [427]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
d1 = reg.sub('',d1,count=1)

# split the table into separate columns
d1 = d1.split('\r\n')
for index in range(0,len(d1)):
    d1[index] = re.split('  +',d1[index])

# load the data into a dataframe
County_household_percent_renters= pd.DataFrame(d1[1:],columns=['County', 'Households', '% Renters'])
County_household_percent_renters

,County,Households,% Renters
0,Barnstable,"96,509",20.0%
1,Berkshire,"53,792",30.0%
2,Bristol,"220,528",37.3%
3,Dukes,"18,146",22.6%
4,Essex,"297,898",36.9%
5,Franklin,"30,927",34.1%
6,Hampden,"179,970",39.5%
7,Hampshire,"60,002",30.7%
8,Middlesex,"612,366",38.1%
9,Nantucket,"11,399",30.8%


In [428]:
#name file 
name=re.sub('\/', '', date)
name=name[50:]
print(name)

week-ending-2020-10-24


In [429]:
#pip install xlsxwriter
#pip install xlwx
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r"C:\Users\wjg39\Documents\Desktop\AP Evictions\\"+name+".xlsx")

In [430]:
#write each DataFrame to a specific sheet
courts.to_excel(writer, sheet_name='Courts',index=False)
party_type.to_excel(writer, sheet_name='Party Type',startrow=0 , startcol=0,index=False)
party_type_pct.to_excel(writer, sheet_name='Party Type',startrow=6 , startcol=0,index=False)
plaintiff_repr.to_excel(writer, sheet_name='Plaintiff Representation',startrow=0 , startcol=0,index=False)
plaintiff_repr_pct.to_excel(writer, sheet_name='Plaintiff Representation',startrow=6 , startcol=0,index=False)
defendant_repr.to_excel(writer, sheet_name='Defendant Representation',startrow=0 , startcol=0,index=False)
defendant_repr_pct.to_excel(writer, sheet_name='Defendant Representation',startrow=6 , startcol=0,index=False)
num_adults.to_excel(writer, sheet_name='Number of Adults in Households',index=False)
init_action.to_excel(writer, sheet_name='Initiating Action',index=False)
rate_per_100k.to_excel(writer, sheet_name='Municipality rate per 100k',index=False)
rate_per_100k_renters.to_excel(writer, sheet_name='County rate per 100k',index=False)
County_household_percent_renters.to_excel(writer, sheet_name='County Households % Renters',index=False)

#close the Pandas Excel writer and output the Excel file
writer.save()